# Sqlite database
<a href="https://colab.research.google.com/github/rambasnet/FDSPython-Notebooks/blob/master/Ch21-SqliteDB.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://www.sqlite.org/
- C-based, one of the most used embedded database (zero configuration)

## sqlite browser
- http://sqlitebrowser.org/

## sql basics
 - http://www.w3schools.com/sql/default.asp

## sqlite3 module
- https://docs.python.org/3/library/sqlite3.html
- SQLite natively supports the following types: NULL, INTEGER, REAL, TEXT, BLOB 
<table>
<tr><td>Python type</td>
    <td>SQLite type</td>
</tr>
<tr><td>None</td>
    <td>NULL</td>
</tr>
<tr><td>int</td>
    <td>INTEGER</td>
</tr>
<tr><td>float</td>
    <td>REAL</td>
</tr>
<tr><td>str</td>
    <td>TEXT</td>
</tr>
<tr><td>bytes</td>
    <td>BLOB</td>
</tr>
</table>

## in memory db example

In [ ]:
import sqlite3
# connect to the memory database
con = sqlite3.connect(":memory:")

# create a table
con.execute("create table person(fname, lname)")

In [ ]:
# fill the table with data
persons = [('Hugo', 'Boss'), ('Calvin', 'Klien')]
con.executemany("insert into person(fname, lname) values (?, ?)", 
                persons)

In [ ]:
# print the table contents
for row in con.execute("select rowid, fname, lname from person"):
    print(row)

In [ ]:
print("I just deleted", con.execute("delete from person where rowid=1").rowcount, 
      "rows")

## db file example
### create database, create table and insert data into table

In [ ]:
import sqlite3
# create connection
conn = sqlite3.connect('example.db')
# create cursor object
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS students (
                firstName text, 
                lastName text, 
                test1 real, 
                test2 real,
                average real,
                grade text
                )
            """)

In [ ]:
query = """ INSERT INTO students (firstName, lastName, 
            test1, test2) values (?, ?, ?, ?)
        """
cur.execute(query, ('John', 'Smith', 99, 95.5))

In [ ]:
cur.execute(query, ('Michael', 'Jordan', 50, 65))

In [ ]:
# save/commit the changes to the db
conn.commit()
# close the database if done
conn.close()

### open database, read and update table

In [ ]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()

In [ ]:
cur.execute('SELECT * FROM students where rowid = 1')
row = cur.fetchone() # returns one row as tuple if rowid with value 1 exists
print(row)

In [ ]:
for col in row:
    print(col)

In [ ]:
cur.execute('SELECT rowid, * FROM students')
rows = cur.fetchall()
print(type(rows))

In [ ]:
for row in rows:
    print(row)

<strong>update table</strong>

In [ ]:
for row in rows:
    avg = (row[3] + row[4])/2
    # grade = ?
    cur.execute('update students set average=? where rowid=?', (avg, row[0]))

In [ ]:
cur.execute('select * from students')
print(cur.fetchall())

In [ ]:
# commit changes and close connection
conn.commit()
conn.close()

## SQL Injection Vulnerability
- how not to write sql query in programs

In [ ]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS users (
                username text unique, 
                password text 
                )
            """)

In [ ]:
# Prompt user to create account
username = input('Enter your username: ')
password = input('Pick a password: ')

In [ ]:
# bad passwords
# insecure way to create sql statements
sqlinsert = "insert into users (username, password) values '{0}', '{1}') \
            ".format(username, password)

cur.execute(sqlinsert)


In [ ]:
# check database
conn.commit()
for row in cur.execute('select * from users'):
    print(row)

### what is wrong with the above codes?

### verify users and SQL injection attack

In [ ]:
# Prompt user to create account
def insecureAuthentication():
    username = input('Enter your username: ')
    password = input('Pick a password: ')
    sqlSelect = "select * from users where username = '{0}' \
                    and password = '{1}'".format(username, password)
    cur.execute(sqlSelect)
    row = cur.fetchone()
    if row:
        print('Welcome {}, this is your kingdom!'.format(row[0]))
    else:
        print('Wrong credentials. Try Again!')
        

In [ ]:
insecureAuthentication()

## secure way to store password
- https://docs.python.org/3/library/hashlib.html


In [ ]:
import uuid
import hashlib, binascii

def createSecurePassword(password, salt=None, round=100000):
    if not salt:
        salt = uuid.uuid4().hex
    
    """
    for i in range(round):
        password = password+salt
        password = hashlib.sha256(password.encode('utf-8')).hexdigest()
    """
    # hashlib.pbkdf2_hmac(hash_name, password, salt, iterations, dklen=None)
    dk = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), 
                        salt.encode('utf-8'), round)
    password = binascii.hexlify(dk)
    return "%s:%s"%(password, salt)

In [ ]:
def secureRegistration():
    # Prompt user to create account
    username = input('Enter your username: ')
    password = input('Enter your password: ')
    secPass = createSecurePassword(password)
    insert = 'insert into users (username, password) values (?, ?)'
    cur.execute(insert, (username, secPass))
    

In [ ]:
# register a user
secureRegistration()

In [ ]:
# check data
for row in cur.execute('select * from users'):
    print(row)

In [ ]:
conn.commit()

In [ ]:
def secureAuthentication():
    username = input('Enter your username: ')
    password = input('Enter your password: ') 
    # use parameterized query
    sqlSelect = 'select password from users where username = ?'
    cur.execute(sqlSelect, (username,))
    row = cur.fetchone()
    if row:
        # username exists
        # check password hashes
        hashpass = row[0]
        hashedPass = hashpass[:hashpass.find(':')]
        salt = hashpass[hashpass.find(':')+1:]
        secPass = createSecurePassword(password, salt)
        if hashpass == secPass:
            print('Welcome to your kingdom, {}'.format(username))
        else:
            print('Wrong credentials. Try Again!')
    else:
        print('Wrong credentials. Try Again!')

In [ ]:
secureAuthentication()

In [ ]:
conn.commit()
conn.close()